# Assignment 6: Classification with Logistic Regression

# Total: 16 Marks + 2 Bonus Marks

## Instructions

* Complete the assignment

* Once the notebook is complete, restart your kernel and rerun your cells

* Submit this notebook to owl by the deadline

## The Dataset

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

You can read more about the data and the variables [here](https://www.kaggle.com/uciml/pima-indians-diabetes-database).

In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler










## Part A: 1 Mark

Read in the `diabetes.csv` dataset.

In [17]:
data=pd.read_csv('diabetes.csv')
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## Part B: 1 Mark

Split the data into train and test.  Hold out 50% of observations as the test set.  Pass `random_state=0` to `train_test_split` to ensure you get the same train and tests sets as the solution.

In [18]:
y = data.BloodPressure
X = data.drop('BloodPressure', axis = 'columns')


Xtrain, Xtest, ytrain, ytest = train_test_split(X,
                                                y, 
                                                test_size=0.5, 
                                                random_state=0)
print(Xtrain.shape,Xtest.shape)

(384, 8) (384, 8)


## Part C: 3 Marks

Read the documentation for [sklearn's `LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).  In no more than 2 sentences per bullet point, answer the following in your own words.

* Does `LogisticRegression` use a penalty by default?  If yes, what penalty?  If it does, does this mean that `LogisticRegression` actually uses ridge regression or the LASSO as the default?
-yes.'LogisticRegreesion' use a penalty by default. It appears to be L2 regularization with a constant of L1 and it will be ridge regression. 


* What is the interpretation of the parmater `C`?  How does it relate to the regularization strength $\lambda$?
-C is 1 divide by lambda and lowering C will strengthen the Lambda regulator. 


* If I were to ask you to use a regularization strengh of 2 (i.e. $\lambda=2$), what value of `C` would you pass?
-the larger the parameter C, the more complex the relationships in the data that the model can recover.10



## Part D: 1 Mark

Create a pipeline for logistic regression (that is, the unpenalized version).  You will need to choose an alternative solver for `LogisticRegression` since the default solver does not support the no penalty option. Any solver will do, so just take a look at the docs to see what is available. I used `solver="newton-cg"` which seems to work fine.

If you get a warning about convergence of `coef_`, try increasing the `max_iter` parameter.  I used `max_iter=10000` which seems to supress the warning.

In [19]:
model_pipeline = Pipeline([
    ('logistic_regression', LogisticRegression(solver="newton-cg"))
])

## Part E: 3 Marks

Use your model to construct a confusion matrix by fitting and predicting on the training data (I've inlcluded a little helper function to make looking at the confusion matrix a little easier). Then answer the following using the confusion matrix (don't use sklearn's functions):

* What is your model's training accuracy?
* What is your model's training precision?
* What is your model's training recall?

In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots()
    ax.axis('equal')
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout();
    return ax

lr = model_pipeline.fit(Xtrain,ytrain)
ytest_pred = lr.predict(Xtest)
plot_confusion_matrix(ytest_pred, ytest, data)


/opt/tljh/user/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


KeyError: "None of [Int64Index([  0,  24,  30,  44,  46,  48,  50,  52,  54,  55,  56,  58,  60,\n             61,  62,  64,  65,  66,  68,  70,  72,  74,  75,  76,  78,  80,\n             82,  84,  85,  86,  88,  90,  92,  94,  95,  96,  98, 100, 102,\n            104, 106, 108, 110, 114],\n           dtype='int64')] are in the [columns]"

## Part F:  1 Mark

Estimate logistic regressions out of sample recall by using 5 fold cross validation.


In [ ]:
cv_score = cross_val_score(model_pipeline, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')

cv_score_mean = -cv_score.mean()

print(cv_score_mean)

# Part F: 2 Marks

Create a new pipeline for either an l2 penalty in logistic regression or an l1 penalty.  Remember, penalized models perform best when you scale the inputs.  You should add `StandardScaler()` to your pipeline.

In [27]:
pipe = Pipeline([
    (('scale', StandardScaler()), LogisticRegression(solver="liblinear"))
])

## Part F: 2 Marks

  Use sklearn's `GridSearchCV` to search over the regulatization strength ranging from 0.01 to 10 in 25 evenly spaced increments for you model.  Use recall as your metric for scoring.
  

`GridSearchCV` is a way to cross validate your models for a variety of parameters.  Read more about `GridSearchCV` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [45]:
from sklearn.svm import SVR
import numpy as np
from sklearn.model_selection import GridSearchCV

sc = StandardScaler()


param_grid = dict(clf__C=np.logspace(0.01,10,25),
                   clf__kernel=['rbf','linear'])
gsc = GridSearchCV(pipe, param_grid, cv =3, n_jobs=1, verbose =2, scoring ='accuracy')

C = np.logspace(0.01,10,25)
#for c in C:
    #clf = LogisticRegression(penalty='l1', C=c, solver='liblinear')
    #clf.fit(Xtrain, ytrain)
   # print('C:', c)
   # print('Coefficient of each feature:', clf.coef_)
   # print('Training accuracy:', clf.score(sc.fit_transform(Xtrain), ytrain))
   # print('Test accuracy:', clf.score(sc.transform(Xtest), ytest))
    #print('')
    
alphas = np.logspace(0.01, 10, 25)
grid = GridSearchCV(estimator=Lasso(),
param_grid=dict(alpha=alphas), cv=10, scoring='r2')

## Part G: 1 Mark

What was the cross validated recall for your regularized model?  If you called your model grid search `lasso_gscv` you can access the best model's score by performing `lasso_gscv.best_score_`.


In [48]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV


alphas = np.logspace(0.01, 10, 25)
grid = GridSearchCV(estimator=Lasso(),
param_grid=dict(alpha=alphas), cv=10, scoring='r2')


print (grid.best_score_)


AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

## Part H:  2 Marks

Print out the `C` parameter for your regualized model.  Answer the following:

* From this value what is the regularization strength $\lambda$?  
-regulrization strength will be 1 in here



* In 1-2 sentences, would you prefer a regularized model for this dataset?  When might a regularized model make more sense?
i would say no because we used the regularization strength of 1 and it will be more make sense when the data is overfitting.

In [52]:
lasso = Lasso(alpha=0.001487)
lasso.fit(Xtrain, ytrain)
print (lasso.score(Xtest, ytest))

model_pipeline




0.11819117538884583


Pipeline(memory=None,
         steps=[('logistic_regression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='newton-cg', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

# BONUS: 2 Marks

The following is a bonus question worth 2 marks. If you do not answer this question, there is no penalty.  You can't earn more than 100% on this assignment.

LASSO and other penalized estimators make the assumption that the model is what we call *sparse* (that means, not every variable is actually related to the outcome).  We can see which variables are more important than others by examining what is known as the *coefficient paths*.

Here are the steps to create the coefficient path:

1) Initialize an array of regularization strengths (typically going from something very small, maybe 0.1, to something very large, maybe 100.

2) For each regularization strength, fit your model.  Keep track of the coefficients.

3) Plot the coefficient values against the log of the regularization strength.


You've done this correctly if you can create a plot which looks like [this](https://cvxpy.readthedocs.io/en/latest/_images/lasso_regression_11_0.svg).

To earn the bonus marks, do the following:

* Construct the coefficient path for logistic regression with an l1 penalty.
* Determine which coefficient is most strongly related to the outcome be examining which coefficent reaches 0 last.


